In [ ]:
# !pip install scapy
# !pip install numpy

In [2]:
import struct
import random
import numpy as np
from scapy.all import *

In [3]:
src_addr = 'localhost'  # Адрес отправителя
dst_addr = '1.2.3.4'    # Адрес получателя

rows = 20  # Количество строк матрицы
cols = 20  # Количество столбцов матрицы

num_interactions = 2000  # Количество запросов
max_num_registers = 25   # Максимальное количество регистров в каждом пакете

In [4]:
def generate_engine_matrix(rows, cols):
    # Создаем матрицу со случайными значениями от 1 до 100
    matrix = np.random.randint(1, 100, size=(rows, cols))
    
    # Добавляем заголовки для столбцов (C0 до C19)
    column_headers = [f"R{i}" for i in range(cols)]
    
    # Добавляем заголовки для строк (R0 до R19)
    row_headers = [f"C{i}" for i in range(rows)]
    
    # Выводим матрицу с заголовками
    print("\t".join([""] + column_headers))
    for i in range(rows):
        print("\t".join([row_headers[i]] + list(map(str, matrix[i]))))

    return matrix

In [5]:
def create_modbus_tcp_packet(unit_id, function_code, data):
    # Преобразование заголовка Modbus/TCP
    protocol_id = 0
    length = len(data) + 2

    # Создание заголовка
    header = struct.pack('>HHHBB', transaction_id, protocol_id, length, unit_id, function_code)

    # Добавление данных
    packet = header + data

    return packet

In [6]:
def create_modbus_response(unit_id, function_code, data):
    return create_modbus_tcp_packet(unit_id, function_code, data)

In [7]:
def create_modbus_request(unit_id, starting_address, quantity):
    data = struct.pack('>HH', starting_address, quantity)
    modbus_request = create_modbus_tcp_packet(unit_id, 3, data)
    return modbus_request

In [8]:
def create_modbus_response_from_matrix(matrix, starting_address, quantity):
    # Преобразование данных из матрицы в формат Modbus Response
    response_data = b''
    for value in matrix.flatten()[starting_address:starting_address + quantity]:
        response_data += struct.pack('>H', value)
    
    byte_count = len(response_data)
    return struct.pack('>B', byte_count) + response_data

In [9]:
# Пример использования для создания Modbus запроса и записи в PCAP файл:
unit_id = 1

# Создание случайной матрицы
matrix = generate_engine_matrix(rows, cols)

packets = []  # Список для хранения пакетов
used_transaction_ids = set()  # Множество для отслеживания использованных transaction_id

	R0	R1	R2	R3	R4	R5	R6	R7	R8	R9	R10	R11	R12	R13	R14	R15	R16	R17	R18	R19
C0	21	66	52	8	78	97	12	6	54	29	19	36	38	89	87	51	46	6	72	63
C1	60	59	96	9	32	12	82	17	71	66	52	98	38	39	37	65	41	57	2	85
C2	44	8	47	73	11	87	55	55	9	70	4	89	45	93	80	25	59	36	24	10
C3	60	47	99	93	8	87	40	15	71	47	96	84	82	48	87	64	2	89	97	26
C4	88	24	38	52	86	50	76	38	6	75	8	19	14	86	54	68	32	77	98	1
C5	2	23	75	2	1	43	25	64	88	5	55	57	9	84	78	18	29	4	12	53
C6	74	77	86	47	38	82	24	21	99	65	59	5	9	47	71	27	75	18	99	37
C7	74	58	98	85	20	81	3	49	10	88	17	41	10	52	80	92	83	75	37	77
C8	75	65	73	19	70	53	2	64	41	54	92	20	42	90	38	48	82	34	29	2
C9	47	65	45	28	50	91	34	23	24	18	48	65	29	74	30	71	1	73	22	77
C10	45	68	87	12	13	73	36	81	7	36	15	73	29	64	31	88	63	86	32	30
C11	60	12	79	61	88	88	48	73	86	65	2	48	73	39	48	74	7	95	90	32
C12	48	9	81	27	27	20	25	87	20	4	64	34	66	95	62	53	21	34	89	90
C13	51	62	74	49	12	64	53	75	11	44	73	80	31	97	46	85	82	96	39	24
C14	28	32	80	81	89	98	84	23	88	29	85	46	91	63	47	84	23	18	25	63
C15	36	80

In [10]:
for _ in range(num_interactions):
    # Генерация случайных значений starting_address и quantity
    starting_address = random.randint(0, (rows * cols - 1))
    quantity = random.randint(1, max_num_registers)

    # Генерация уникального transaction_id
    while True:
        transaction_id = random.randint(0, num_interactions)
        if transaction_id not in used_transaction_ids:
            used_transaction_ids.add(transaction_id)
            break

    # Генерация случайного порта
    random_port = random.randint(1024, 65535)

    # Создание Modbus запроса
    modbus_request = create_modbus_request(unit_id, starting_address, quantity)

    # Преобразование в IP/TCP пакет с добавлением случайного порта назначения
    ip_packet_request = IP(src=src_addr, dst=dst_addr) / TCP(dport=502, sport=random_port) / modbus_request

    # Добавление в список
    packets.append(ip_packet_request)

    # Обработка запроса и создание Modbus Response
    modbus_response_data = create_modbus_response_from_matrix(matrix, starting_address, quantity)
    modbus_response = create_modbus_response(unit_id, 3, modbus_response_data)

    # Преобразование в IP/TCP пакет с добавлением случайного порта назначения
    ip_packet_response = IP(src=dst_addr, dst=src_addr) / TCP(dport=random_port, sport=502) / modbus_response

    # Добавление в список
    packets.append(ip_packet_response)

In [11]:
# Запись в PCAP файл
pcap_filename = 'modbus_interactions.pcap'

wrpcap(pcap_filename, packets)
pcap_p = rdpcap(pcap_filename)
pcap_p[0]

<IP  version=4 ihl=5 tos=0x0 len=52 id=1 flags= frag=0 ttl=64 proto=tcp chksum=0xf7bc src=127.0.0.1 dst=1.2.3.4 |<TCP  sport=29502 dport=502 seq=0 ack=0 dataofs=5 reserved=0 flags=S window=8192 chksum=0x93cd urgptr=0 |<Raw  load='\x01\\xda\x00\x00\x00\x06\x01\x03\x00\\xea\x00\x01' |>>>